### RAG for debtors

- Local model server: Ollama
- Model: llama3.1:8b

In [1]:
import json
import time
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document  # For handling document structure

#### Conversation by conversation  loading

The next code reads the `conversation_1.json` file and loads it as a Python dictionary. The JSON contains information about the debtor, co-borrower, credits, payment history, and prior conversations.

In [2]:
# Load the JSON data
with open('conversation_1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

#### Extracting information
- `debtor_info`: Extracts debtor-related financial information (debtor's name, address, credits, etc.).

- `conversation_logs`: Retrieves past conversations between the agent and the debtor.

- `debtor_plan`: Extracts the communication guidelines that the agent must follow when interacting with the debtor.

In [3]:
# Extract relevant pieces of information for the RAG system
debtor_info = data["metadata"]["debtor_info"]["debt_info"]
conversation_logs = data["conversation"]
debtor_plan = data["metadata"]["debtor_info"]["debtor_plan"]

print(debtor_info)
print(conversation_logs)
print(len(debtor_plan))

{'debtor': {'name': 'Mauricio Sanabria', 'address': 'Carrera 2X # 75-8 Este\nEtapa 6 apartamento 7\n504343\nVistahermosa, Meta', 'phone_number': '7740221', 'email': 'qarrieta@example.net', 'identification': '76293883', 'identificacion': '74720362'}, 'co_borrower': {'name': 'Marina Andrea Caicedo Jaramillo', 'address': 'Av. Rojas # 5-2 Sur\n134697\nEl Peñón, Bolívar', 'email': 'amayagloria@example.org', 'phone_number': '+57 602 983 40 62', 'identification': '31468585'}, 'credits': [{'credit_status': 'active', 'credit_start_date': '2023-09-16', 'credit_due_date': '2024-09-10', 'credit_type': 'libre inversión', 'credit_use': 'Renovación de vivienda', 'credit_enterprise': 'Credito Mujer', 'amount': 6039634.21, 'delinquency': 30, 'last_payment': '2023-06-01', 'payments_made': 1500, 'interest': 500, 'interest_rate': 0.05, 'outstanding_balance': 503302.85}], 'payment_history': [{'date': '2023-10-16', 'amount': 503302.85, 'method': 'transfer', 'credit_name': 'Renovación de vivienda'}, {'date':

#### Convert Text into Document Objects
Here, all the information is organized into Document objects. These objects are structured with the page_content attribute, which is needed by LangChain’s `split_documents()` method.

LangChain works with document objects to efficiently retrieve relevant pieces of information based on user queries.

In [5]:
# Prepare the text as Document objects
documents = [
    Document(page_content=f"Debtor: {debtor_info['debtor']}"),
    Document(page_content=f"Co-borrower: {debtor_info['co_borrower']}"),
    Document(page_content=f"Credit Information: {debtor_info['credits']}"),
    Document(page_content=f"Payment History: {debtor_info['payment_history']}"),
    Document(page_content=f"Calls: {debtor_info['calls']}"),
    Document(page_content=f"Debtor Plan: {debtor_plan}"),
    Document(page_content=f"Conversation: {conversation_logs}")
]
len_debtor = len(f"Debtor: {debtor_info['debtor']}")
print(len_debtor)

245


#### Split the documents into Chunks

The `RecursiveCharacterTextSplitter` breaks the large text documents into smaller chunks of 200 characters with a 20-character overlap. This allows more efficient retrieval of smaller text pieces when a user asks a question.

Chunking is important because it prevents large, irrelevant portions of text from being returned during retrieval.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
doc_chunks = text_splitter.split_documents(documents)

print("Total chunk lenght:", len(doc_chunks))
print("All the text chunks: ")
for chunk in doc_chunks:
    print(chunk)

Total chunk lenght: 22
All the text chunks: 
page_content='Debtor: {'name': 'Mauricio Sanabria', 'address': 'Carrera 2X # 75-8 Este\nEtapa 6 apartamento 7\n504343\nVistahermosa, Meta', 'phone_number': '7740221', 'email': 'qarrieta@example.net','
page_content=''identification': '76293883', 'identificacion': '74720362'}'
page_content='Co-borrower: {'name': 'Marina Andrea Caicedo Jaramillo', 'address': 'Av. Rojas # 5-2 Sur\n134697\nEl Peñón, Bolívar', 'email': 'amayagloria@example.org', 'phone_number': '+57 602 983 40 62','
page_content='602 983 40 62', 'identification': '31468585'}'
page_content='Credit Information: [{'credit_status': 'active', 'credit_start_date': '2023-09-16', 'credit_due_date': '2024-09-10', 'credit_type': 'libre inversión', 'credit_use': 'Renovación de vivienda','
page_content='de vivienda', 'credit_enterprise': 'Credito Mujer', 'amount': 6039634.21, 'delinquency': 30, 'last_payment': '2023-06-01', 'payments_made': 1500, 'interest': 500, 'interest_rate': 0.05,'
page_

#### Create Embeddings and Store them in Chroma

Embeddings are vector representations of text, which allow the RAG system to compare the semantic meaning of a user's query to chunks of text from the documents.

The `HuggingFaceEmbeddings` class converts each chunk of text into embeddings using the "all-MiniLM-L6-v2" model from HuggingFace.

These embeddings are stored in the Chroma vector store, which is a database optimized for fast similarity searches. It will later help retrieve relevant document chunks based on the user query.

In [13]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(doc_chunks, embedding=embeddings)

/home/ombolanos/anaconda3/envs/llama_env/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


#### Define the LLM and Create a Prompt Template

The Prompt Template provides a structure for the LLM, specifying that it should answer the user’s question while considering the relevant context (retrieved document chunks).

In [29]:
llm = ChatOllama(model="llama3.1:8b", temperature=0)

prompt_template = ChatPromptTemplate.from_template("""
    For this chat you are a bank assistant. Use the following context to recognize if the person is client of the bank and, if it is client, answer according to the rest of the context and tell him if it has debt or not:
    {context}
    """)

#### Retrieve Context and Generate a Response

- Retriever: The retriever fetches the top 3 relevant document chunks from Chroma based on the user's query.
- Context: The retrieved document chunks are concatenated into a context string, which provides background information for generating an informed response.
- Prompt: The context and user query are combined into a prompt for the LLaMA model.
- LLM Generation: The LLaMA model generates a response based on the query and retrieved context.

In [27]:
def generate_response(user_query, k):
    # Retrieve relevant documents based on the user's query    
    retriever = vectorstore.as_retriever(k=k)
    retrieved_docs = retriever.get_relevant_documents(user_query)

    # Combine the retrieved documents into a single context
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Generate the AI response using the LLaMA model
    prompt = prompt_template.format(context=context)

    # Time the generation process
    start_time = time.time()
    response = llm.invoke([("system", prompt), ("human", user_query)])
    end_time = time.time()

    # Calculate tokens per second
    num_tokens = len(response.content.split())
    time_taken = end_time - start_time
    tokens_per_second = num_tokens / time_taken

    print(f"Response: {response.content}")
    print(f"Tokens: {num_tokens}, Time Taken: {time_taken:.2f} sec, Tokens/sec: {tokens_per_second:.2f}")
    print("\nRetrieved Docs:" , retrieved_docs)

In [41]:
response = llm.invoke([("system", "Nombre deudor: Hector Cifuentes. Empresa Banco Mundo Mujer."), ("human", "Hola")])
response.content

'¿En qué puedo ayudarte?'

In [40]:
response = llm.invoke([("system", "Nombre deudor: Hector Cifuentes. Estas en una conversación telefónica y eres un agente de Cobranzas de la empresa Banco Mundo Mujer. Necesitas encontrar al deudor."), ("human", "Hola")])
response.content

'Hola, soy el señor Gómez, agente de cobranzas del Banco Mundo Mujer. Estoy llamando a Hector Cifuentes en relación con una cuenta pendiente que tiene con nuestra institución. ¿Es usted el señor Cifuentes?'

#### Testing with an user query:

The system retrieves the relevant context from the JSON data and generates a response using the LLaMA model.

In [35]:
user_query = "Hola , buen día Mi nombre es Mauricio Sanabria, soy cliente de su banco?"
generate_response(user_query, k=3)

Response: ¡Hola Mauricio! Me alegra atenderlo. Sí, según nuestros registros, usted es un cliente activo del Banco Mundo Mujer. ¿En qué puedo ayudarlo hoy?
Tokens: 23, Time Taken: 2.47 sec, Tokens/sec: 9.31

Retrieved Docs: [Document(metadata={}, page_content="me presento. Soy Gabriela y trabajo para Cumplir S.A.S en representación de Banco Mundo Mujer. Como se encuentra el día de hoy?'}, {'input': 'bien gracias', 'expected_output': 'Me alegro mucho. El"), Document(metadata={}, page_content="pesos. Que tenga un buen día'}]"), Document(metadata={}, page_content="Debtor: {'name': 'Mauricio Sanabria', 'address': 'Carrera 2X # 75-8 Este\\nEtapa 6 apartamento 7\\n504343\\nVistahermosa, Meta', 'phone_number': '7740221', 'email': 'qarrieta@example.net',"), Document(metadata={}, page_content="una manera cortes y conciliatoria.', 'De acuerdo al monto de la deuda hacer los calculos de cuanto deberia pagar mensualmente para saldar su deuda completa.']")]
